In [45]:
import pandas as pd
from operator import itemgetter
import re
import pymongo

In [46]:
file = "state_tier1_caps.csv"

#Read csv into Pandas
df = pd.read_csv(file)
df.head()

,Emissions are in 1000 tons,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,STATE_FIPS,STATE_ABBR,tier1_code,tier1_description,pollutant_code,emissions90,emissions96,emissions97,emissions98,emissions99,...,emissions09,emissions10,emissions11,emissions12,emissions13,emissions14,emissions_fixed,emissions15,emissions16,emissions17
1,01,AL,01,FUEL COMB. ELEC. UTIL.,CO,6.86859,8.06884,8.04705,8.12158,11.51526,...,10.9668444,10.4941125,9.9576446,9.6423108,9.326977,9.0116432,9.0116432,9.0116432,9.0116432,9.0116432
2,01,AL,01,FUEL COMB. ELEC. UTIL.,NH3,NaN,0.01516,0.01605,0.01597,0.01651,...,0.522571961,0.505945589,0.489319218,0.488462596,0.487605975,0.486749353,0.486749353,0.486749353,0.486749353,0.486749353
3,01,AL,01,FUEL COMB. ELEC. UTIL.,NOX,187.79386,219.009,209.98918,198.66722,186.3871,...,96.00923043,78.87946547,61.6871485,57.76831163,53.84947477,49.9306379,49.9306379,44.893997,27.631909,21.307058
4,01,AL,01,FUEL COMB. ELEC. UTIL.,PM10,8.85994,9.28102,8.83472,8.37946,29.21702,...,5.876249051,6.615406896,7.323321741,6.469981304,5.616640867,4.763300429,4.763300429,4.763300429,4.763300429,4.763300429


In [47]:
# Drop rows where pollutant_code (Unnamed: 4) is not CO

df.drop(df[df['Unnamed: 4'] != 'CO'].index,inplace=True)
df.head()

,Emissions are in 1000 tons,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
1,01,AL,01,FUEL COMB. ELEC. UTIL.,CO,6.86859,8.06884,8.04705,8.12158,11.51526,...,10.9668444,10.4941125,9.9576446,9.6423108,9.326977,9.0116432,9.0116432,9.0116432,9.0116432,9.0116432
8,01,AL,02,FUEL COMB. INDUSTRIAL,CO,25.73799,49.77487,48.15407,48.41578,43.88043,...,22.61858405,22.84104926,71.86470057,66.7471092,61.62951784,56.51192647,56.51192647,56.51192647,56.51192647,56.51192647
15,01,AL,03,FUEL COMB. OTHER,CO,89.93378,34.23424,34.29082,34.15163,65.515923,...,11.54395555,11.75324656,12.10373583,14.99790364,17.89207145,20.78623926,20.78623926,20.78623926,20.78623926,20.78623926
22,01,AL,04,CHEMICAL & ALLIED PRODUCT MFG,CO,54.70092,44.2331,45.59779,46.96713,7.85382,...,1.736527033,2.429740967,3.122954901,3.133777588,3.144600276,3.155422963,3.155422963,3.155422963,3.155422963,3.155422963
29,01,AL,05,METALS PROCESSING,CO,6.61641,36.1217,37.64911,36.8687,31.76794,...,17.58291247,14.28719843,10.9914844,11.17310068,11.35471695,11.53633323,11.53633323,11.53633323,11.53633323,11.53633323


In [48]:
# Rename columns
#df.rename(columns={'Leader': 'Commander'}, inplace=True)
df.rename(columns={'Unnamed: 1': 'State_Abbreviation', "Unnamed: 4":"Poluttant_Code", "Unnamed: 23":"2012_Emissions",
                  "Unnamed: 24":"2013_Emissions", "Unnamed: 25":"2014_Emissions", "Unnamed: 26":"2015_Emissions",
                  "Unnamed: 27":"2016_Emissions", "Unnamed: 28":"2017_Emissions"}, inplace=True)
          
df.head()

,Emissions are in 1000 tons,State_Abbreviation,Unnamed: 2,Unnamed: 3,Poluttant_Code,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,2012_Emissions,2013_Emissions,2014_Emissions,2015_Emissions,2016_Emissions,2017_Emissions
1,01,AL,01,FUEL COMB. ELEC. UTIL.,CO,6.86859,8.06884,8.04705,8.12158,11.51526,...,10.9668444,10.4941125,9.9576446,9.6423108,9.326977,9.0116432,9.0116432,9.0116432,9.0116432,9.0116432
8,01,AL,02,FUEL COMB. INDUSTRIAL,CO,25.73799,49.77487,48.15407,48.41578,43.88043,...,22.61858405,22.84104926,71.86470057,66.7471092,61.62951784,56.51192647,56.51192647,56.51192647,56.51192647,56.51192647
15,01,AL,03,FUEL COMB. OTHER,CO,89.93378,34.23424,34.29082,34.15163,65.515923,...,11.54395555,11.75324656,12.10373583,14.99790364,17.89207145,20.78623926,20.78623926,20.78623926,20.78623926,20.78623926
22,01,AL,04,CHEMICAL & ALLIED PRODUCT MFG,CO,54.70092,44.2331,45.59779,46.96713,7.85382,...,1.736527033,2.429740967,3.122954901,3.133777588,3.144600276,3.155422963,3.155422963,3.155422963,3.155422963,3.155422963
29,01,AL,05,METALS PROCESSING,CO,6.61641,36.1217,37.64911,36.8687,31.76794,...,17.58291247,14.28719843,10.9914844,11.17310068,11.35471695,11.53633323,11.53633323,11.53633323,11.53633323,11.53633323


In [50]:
df.drop(["Emissions are in 1000 tons", "Unnamed: 2", "Unnamed: 3", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 12",
        "Unnamed: 13", "Unnamed: 14", "Unnamed: 15", "Unnamed: 16", "Unnamed: 17", "Unnamed: 18", "Unnamed: 19", "Unnamed: 20",
        "Unnamed: 21", "Unnamed: 22",], axis=1, inplace=True)

df.head()

,State_Abbreviation,Poluttant_Code,2012_Emissions,2013_Emissions,2014_Emissions,2015_Emissions,2016_Emissions,2017_Emissions
1,AL,CO,9.326977,9.0116432,9.0116432,9.0116432,9.0116432,9.0116432
8,AL,CO,61.62951784,56.51192647,56.51192647,56.51192647,56.51192647,56.51192647
15,AL,CO,17.89207145,20.78623926,20.78623926,20.78623926,20.78623926,20.78623926
22,AL,CO,3.144600276,3.155422963,3.155422963,3.155422963,3.155422963,3.155422963
29,AL,CO,11.35471695,11.53633323,11.53633323,11.53633323,11.53633323,11.53633323


In [ ]:
df.dtypes

In [52]:
# df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)

df[["2012_Emissions", "2013_Emissions", "2014_Emissions", 
    "2015_Emissions", "2016_Emissions", "2017_Emissions"]] = df[["2012_Emissions", "2013_Emissions", "2014_Emissions", 
                                                                "2015_Emissions", "2016_Emissions", "2017_Emissions"]].apply(pd.to_numeric)

df.dtypes

State_Abbreviation     object
Poluttant_Code         object
2012_Emissions        float64
2013_Emissions        float64
2014_Emissions        float64
2015_Emissions        float64
2016_Emissions        float64
2017_Emissions        float64
dtype: object

In [53]:
df.head()

,State_Abbreviation,Poluttant_Code,2012_Emissions,2013_Emissions,2014_Emissions,2015_Emissions,2016_Emissions,2017_Emissions
1,AL,CO,9.326977,9.011643,9.011643,9.011643,9.011643,9.011643
8,AL,CO,61.629518,56.511926,56.511926,56.511926,56.511926,56.511926
15,AL,CO,17.892071,20.786239,20.786239,20.786239,20.786239,20.786239
22,AL,CO,3.144600,3.155423,3.155423,3.155423,3.155423,3.155423
29,AL,CO,11.354717,11.536333,11.536333,11.536333,11.536333,11.536333


In [ ]:
# hosted_in_us = reduced_kickstarter_df.loc[reduced_kickstarter_df["country"] == "US"]

# Iterate through df to get separate states

#     db_input = []
#     for index, row in chk_df.iterrows():
#         state_record = {}
#         for i in COLUMNS:
#             state_record[i] = row[i]
#         db_input.append(state_record)

